# We are going to use unstructured and structured data in our data stores to answer our query
# This is possible via an entity called an **agent**.

In simplest terms, agents are `LLM Powered Knowledge workers`

We are going to get the unstructured data from **wikipedia** and unstructured from the database (which will contain ratings and other quantitative stuff) we will create using a csv.

Install Pre-requisites

In [ ]:
!pip install -U -q nest_asyncio openai llama-index llama-index-embeddings-nomic llama-index-readers-wikipedia llama-index-llms-gradient nltk tiktoken sentence-transformers pydantic wikipedia sqlalchemy pandas python-dotenv

Notebook doesn't allow async operations to complete properly when applied in a loop. So we will use nest_asyncio for it

In [ ]:
import nest_asyncio
import logging
import sys

nest_asyncio.apply()

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

INITIATE OPENAI KEYS

In [ ]:
import os
from pprint import pformat
from dotenv import dotenv_values
env_file_path = "env.txt"

env_dict = dotenv_values(env_file_path)

WE ARE GOING TO USE NOMIC EMBEDDING MODEL FOR THIS TASK FOR WHICH A VERY NICE [CONNECTOR IS PROVIDED BY llama-index team](https://docs.llamaindex.ai/en/latest/examples/embeddings/nomic/).

In [ ]:
from llama_index.embeddings.nomic import NomicEmbedding


# Nomic has released two models. v1 has fixed dimensionality and v1.5 supports variable dimen
# - nomic-embed-text-v1   | fixed dimensionality
# - nomic-embed-text-v1.5 | variable length dimensionality via matryoshka learning | size range : 64 to 768

embedding_model = NomicEmbedding(
    model_name="nomic-embed-text-v1",
    api_key=env_dict['NOMIC_API_KEY']
)

Lets test the embedding model

In [ ]:
embedding_model.get_text_embedding("Nomic Embedding !")

#### Core Settings | Configurations

LlamaIndex has the ability to set `Settings` (Successor of `ServiceContext`). The basic idea here is that we use this to establish some core properties and then can pass it to various services.

While we could set this up as a global, we're going to leave it as `Settings` so we can see where it's applied.

We'll set a few significant contexts:

- `chunk_size` - this is what it says on the tin
- `llm` - this is where we can set what model we wish to use as our primary LLM when we're making `QueryEngine`s and more
- `embed_model` - this will help us keep our embedding model consistent across use cases


We'll also create some resources we're going to keep consistent across all of our indices today.

- `text_splitter` - This is what we'll use to split our text, feel free to experiment here
- `SimpleNodeParser` - This is what will work in tandem with the `text_splitter` to parse our full sized documents into nodes.

In [ ]:
from llama_index.llms.gradient import GradientBaseModelLLM  # import Model
from llama_index.core import Settings                       # import Settings
from llama_index.core.node_parser import SentenceSplitter   # import nodeparser variant
from llama_index.core.llms import ChatMessage
from llama_index.core.callbacks import CallbackManager, LlamaDebugHandler
from llama_index.core.output_parsers import PydanticOutputParser
from pprint import pformat

# using llama3-bb-chat as openai api proxy not working anymore
gradient_handler = LlamaDebugHandler()
gradient_callback = CallbackManager([gradient_handler])

base_model_slug = "llama3-8b-chat"
language_model = GradientBaseModelLLM(
    workspace_id=env_dict['GRADIENT_WORKSPACE_ID'],
    access_token=env_dict['GRADIENT_ACCESS_TOKEN'],
    base_model_slug=base_model_slug,
    max_tokens=300, is_chat_model=True,
    callback_manager=gradient_callback
)

# configure Settings
Settings.llm = language_model
Settings.embed_model = embedding_model
Settings.node_parser = SentenceSplitter(chunk_size=512, chunk_overlap=20)
Settings.context_window = 3900


LETS INITIATE A VECTOR STORE FOR OUR DOCS

- We will get the documents from wikipedia, chunk them and store them in
our vector store

In [ ]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents([])

READ DATA FROM Wikipedia

Setting `auto_suggest=False` ensures we run into fewer auto-correct based errors.

In [ ]:
from llama_index.readers.wikipedia import WikipediaReader

movie_list = ["Barbie (film)", "Oppenheimer (film)"]

wiki_docs = WikipediaReader().load_data(pages=movie_list, auto_suggest=False)

Now we will loop through our documents and metadata and construct nodes (associated with particular metadata for easy filtration later).

In [ ]:
for movie, wiki_doc in zip(movie_list, wiki_docs):
    nodes = Settings.node_parser.get_nodes_from_documents([wiki_doc])

    # add metadata to each node
    for node in nodes:
        node.metadata = {"title": movie}
    index.insert_nodes(nodes=nodes)

#### Auto Retriever Functional Tool

This tool will leverage OpenAI's functional endpoint to select the correct metadata filter and query the filtered index - only looking at nodes with the desired metadata.

A simplified diagram: ![image](https://i.imgur.com/AICDPav.png)

First, we need to create our `VectoreStoreInfo` object which will hold all the relevant metadata we need for each component (in this case title metadata).

Notice that you need to include it in a text list.

Then we will define a retriever `VectorIndexAutoRetriever` which will retrieve relevant info from index.
And finally we will create a retriever_query_engine `RetieverQueryEngine` which will act as us communication point between query and retriever.

In [ ]:
from llama_index.core.tools import FunctionTool
from llama_index.core.vector_stores.types import (
    VectorStoreInfo, MetadataInfo, ExactMatchFilter, MetadataFilters
)
from llama_index.core.retrievers import VectorIndexAutoRetriever
from llama_index.core.query_engine import RetrieverQueryEngine

top_k = 6

# provide vector store information
vector_store_info = VectorStoreInfo(
    content_info="semantic information about movies",
    metadata_info=[MetadataInfo(
        name="title",
        type="str",
        description="one of the movie from the list [Barbie (film), Oppenheimer (film)]"
    )]
)

# make sure that filter terms are an exact match
filter_key_list = ["title"]
filter_value_list = movie_list.copy()

exact_match_filters = [
    ExactMatchFilter(key=k, value=v)
    for k, v in zip(filter_key_list, filter_value_list)
]

# design a retriever which will check index, get top_k while applying filters
vector_auto_retriever = VectorIndexAutoRetriever(
    index=index, vector_store_info=vector_store_info, similarity_top_k=top_k,
    extra_filters=MetadataFilters(filters=exact_match_filters),
    verbose=True
)

# create an engine which will receive the input from the tool
retriever_query_engine = RetrieverQueryEngine.from_args(
    retriever=vector_auto_retriever
)

Here we will define the `QueryEngineTool` for our vector_query_engine which will be provided to our `Agent` as a tool

In [ ]:
from llama_index.core.tools import QueryEngineTool


# vector_tool detailed information
vector_tool_description = f"""
Use this tool to look up semantic information about films.
The vector database schema is given below:
{vector_store_info.json()}
"""
vector_tool = QueryEngineTool.from_defaults(
    query_engine=retriever_query_engine,
    name="vector_tool",
    description=vector_tool_description
)

#### Now we will work to create a `QueryEngineTool` for our vector indexed data

In [ ]:
from llama_index.agent.openai import OpenAIAgent
from llama_index.core.agent import ReActAgent

vector_agent = ReActAgent.from_tools(
    tools=[vector_tool],
    verbose=True,
    max_iterations=20
)

In [ ]:
response = vector_agent.chat(message="Tell me the story of the Barbie movie in 200 words.")

Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: vector_tool
Action Input: {'input': 'Barbie (film)'}


Failed to parse query spec, using defaults as fallback.
Using query str: Barbie (film)
Using filters: []
Observation: The 2023 fantasy comedy film directed by Greta Gerwig, starring Margot Robbie as Barbie and Ryan Gosling as Ken, has received widespread critical acclaim and has become a commercial success. The film's marketing campaign was extensive, with partnerships and collaborations with various brands, and it premiered at the Shrine Auditorium in Los Angeles before being released in the United States and the United Kingdom on July 21, 2023. The film has grossed over $1.446 billion worldwide, making it the highest-grossing film of 2023 and the highest-grossing film ever released by Warner Bros. It has also become the highest-grossing live-action comedy film of all time, surpassing the domestic and worldwide records previously held by Home Alone (1990) and The Hangover Part II (2011) respectively.
Thought: The current language of the user is: English. I need to use a tool to help m

Failed to parse query spec, using defaults as fallback.
Using query str: Oppenheimer (film)
Using filters: []
Observation: The film Oppenheimer, a biographical film about J. Robert Oppenheimer, was released on the same day as Barbie, a fantasy comedy film. The two films appealed to different audiences, leading to a trend dubbed "Barbenheimer."
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: vector_tool
Action Input: {'input': 'Tell me the story of the Barbie movie in 200 words.'}


Failed to parse query spec, using defaults as fallback.
Using query str: Tell me the story of the Barbie movie in 200 words.
Using filters: []
Observation: In a world where Barbie dolls come to life, the iconic fashion doll, Barbie, played by Margot Robbie, is on a journey of self-discovery. She and her partner, Ken, played by Ryan Gosling, navigate the fantastical world of Barbieland and the real world, grappling with existential crises and societal pressures. The film is a commentary on patriarchy and the effects of feminism, as Barbie and her friends, including America Ferrera, Michael Cera, and Kate McKinnon, challenge the norms of their society.

As Barbie struggles to find her identity, she meets the spirit of Ruth Handler, the creator of Barbie, who encourages her to forge her own path. Along the way, Barbie and Ken confront their own biases and learn to appreciate each other's autonomy. The film features a star-studded cast, including Will Ferrell as the CEO of Mattel, and a so

In [ ]:
response.response

"The story of the Barbie movie is about Barbie, played by Margot Robbie, who comes to life and embarks on a journey of self-discovery in a fantastical world. She navigates societal pressures and existential crises alongside her partner Ken, played by Ryan Gosling, and her friends. The film is a commentary on patriarchy and feminism, as Barbie and her friends challenge the norms of their society. Along the way, Barbie meets the spirit of Ruth Handler, the creator of Barbie, who encourages her to forge her own path. The movie features a star-studded cast and a soundtrack that blends classic and modern hits, and tells a story of empowerment, self-discovery, and embracing one's true identity."

### ADDING SQL TO THE AGENT's ARSENAL

The next few steps should be largely straightforward, we'll want to:

1. Read in our `.csv` files into `pd.DataFrame` objects
2. Create an in-memory `sqlite` powered `sqlalchemy` engine
3. Cast our `pd.DataFrame` objects to the SQL engine
4. Create an `SQLDatabase` object through LlamaIndex
5. Use that to create a `QueryEngineTool` that we can interact with through the `NLSQLTableQueryEngine`!

In [ ]:
import pandas as pd

barbie_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/training_data/barbie.csv")
oppenheimer_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/training_data/barbie.csv")

In [ ]:
from sqlalchemy import create_engine

engine = create_engine("sqlite://")

In [ ]:
barbie_df.to_sql(
    "barbie",
    engine
)

oppenheimer_df.to_sql(
    "oppenheimer",
    engine
)

125

In [ ]:
db_tables = ["barbie", "oppenheimer"]

Create SQLDatabase for the sqlengine

In [ ]:
from llama_index.core import SQLDatabase

sql_database = SQLDatabase(
    engine=engine,
    include_tables=db_tables
)

Create the NLSQLTableQueryEngine interface for all added SQL tables

In [ ]:
from llama_index.core.query_engine import NLSQLTableQueryEngine

sql_query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database, verbose=True,
    tables=db_tables
    )

Define a QueryEngineTool which will utilize the sql data we put together.

In [ ]:
sql_tool = QueryEngineTool.from_defaults(
    query_engine=sql_query_engine,
    name="sql_tool",
    description=(
        "Useful for translating a natural language query into a SQL query over"
        " barbie, containing information related to reviews of the Barbie movie"
        " oppenheimer, containing information related to reviews of the Oppenheimer movie. "
        "\n\n".join([
            f"The schema of the table '{table}' is - \n {sql_database.get_table_columns(table)}"
            for table in
            db_tables
        ])
    ),
)

Create an Agent

In [ ]:
sql_agent = ReActAgent.from_tools(
    tools=[sql_tool],
    verbose=True,
    max_iterations=20
)

Testing the response from sql tool

In [ ]:
response = sql_agent.chat("What is the average rating of the two films?")

Thought: The user is asking about the average rating of the two films, Barbie and Oppenheimer. I need to use a tool to help me answer the question.
Action: sql_tool
Action Input: {'input': 'SELECT AVG(Rating) FROM barbie UNION SELECT AVG(Rating) FROM oppenheimer'}
Observation: The average rating of Barbie and Oppenheimer is 7.36.
Thought: The user has provided the average rating of the two films, Barbie and Oppenheimer. I need to verify the accuracy of the answer.
Action: sql_tool
Action Input: {'input': 'SELECT AVG(Rating) FROM barbie UNION SELECT AVG(Rating) FROM oppenheimer'}
Observation: The average rating of Barbie and Oppenheimer is 7.36.
Thought: The user has already provided the average rating of the two films, Barbie and Oppenheimer. I can answer the question without using any more tools. I'll use the user's language to answer.
Answer: The average rating of the two films, Barbie and Oppenheimer, is 7.36.


In [ ]:
response.response

'The average rating of the two films, Barbie and Oppenheimer, is 7.36.'

### FINALLY COMBINING THE TWO TOOLS

In [ ]:
barbenheimer_agent = ReActAgent.from_tools(
    tools=[sql_tool, vector_tool],
    verbose=True,
    max_iterations=100,
    # context="Break down the query into very small steps and then execute them one by one to reach the answer. If you encounter any error, try again by rectifying the error and the approach taken previously"
)

In [ ]:
response = barbenheimer_agent.chat("What is the lowest rating of these films and what did the reviewer said for these ratings?")

Thought: The current language of the user is English. I need to use a tool to help me answer the question.
Action: sql_tool
Action Input: {'input': "SELECT * FROM barbie WHERE Rating = (SELECT MIN(Rating) FROM barbie) AND Review_Title LIKE '%Barbie%' UNION SELECT * FROM oppenheimer WHERE Rating = (SELECT MIN(Rating) FROM oppenheimer) AND Review_Title LIKE '%Oppenheimer%'"}
Observation: It looks like the SQL query is not valid. The error message is not providing much information, but I can try to help you identify the issue.

The query is trying to combine two subqueries using the `UNION` operator. The subqueries are selecting the minimum rating and review title from the `barbie` and `oppenheimer` tables, respectively, where the review title contains the words "Barbie" or "Oppenheimer".

One possible issue is that the subqueries are not properly aliased. The `AS` keyword is used to give an alias to a table or subquery, but it's not used correctly in this query. The correct syntax would 

In [ ]:
response.response

'The lowest rating of the films is 1 (None), and the reviewer\'s summary is: "I really enjoyed the first 20 minutes of the movie. It was very upbeat, positive and light. However, things soon became negative about 20 - 25 minutes into the movie. I couldn\'t stand it. Barbie\'s world and attitude was a constant bummer. Ken\'s attitude was sour. For the next hour (and more) the movie was just a disappointment." This review suggests that the reviewer was initially pleased with the movie, but their opinion quickly turned negative due to the tone and characters.'